# **INTRODUCTIONS**

---

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **NURSING_HOME - NL**

---

## Tables

Archive Table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126]'))
CREATE TABLE [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126] (
	[ID] [int] NOT NULL,
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[INFECTED_NURSERY_DAILY] [int] NULL,
	[DECEASED_NURSERY_DAILY] [int] NULL,
	[TOTAL_NEW_REPORTED_LOCATIONS] [int] NULL,
	[TOTAL_REPORTED_LOCATIONS] [int] NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_RANGE_START] [datetime] NULL,
	[DATE_OF_REPORTS_LAG] [datetime] NULL,
	[DATE_RANGE_START_LAG] [datetime] NULL,
	[7D_AVERAGE_TOTAL_INFECTED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_INFECTED_LAG] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED_LAG] [decimal](16, 2) NULL
);
GO


In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126]'))
CREATE TABLE [VWSARCHIVE].[NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126] (
	[VRCODE] [varchar](50) NOT NULL,
	[VRNAAM] [varchar](100) NOT NULL,
	[VERPLEEGHUIZEN] [float] NOT NULL,
	[GEHANDICAPTENZORGINSTELLINGEN] [float] NOT NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO


Stored procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_NURSING_HOMES_TOTALS_ARCHIVED_20230126] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126]

    INSERT INTO [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126] (
            [ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [INFECTED_NURSERY_DAILY],
            [DECEASED_NURSERY_DAILY],
            [TOTAL_NEW_REPORTED_LOCATIONS],
            [TOTAL_REPORTED_LOCATIONS],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    )
    SELECT 
            [ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [INFECTED_NURSERY_DAILY],
            [DECEASED_NURSERY_DAILY],
            [TOTAL_NEW_REPORTED_LOCATIONS],
            [TOTAL_REPORTED_LOCATIONS],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    FROM [VWSDEST].[NURSING_HOMES_TOTALS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NURSING_HOMES_TOTALS])
END


In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_STATIC_NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126]

    INSERT INTO [VWSARCHIVE].[NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126] (
        [VRCODE],
        [VRNAAM],
        [VERPLEEGHUIZEN],
        [GEHANDICAPTENZORGINSTELLINGEN],
        [DATE_LAST_INSERTED]
    )
    SELECT 
        [VRCODE],
        [VRNAAM],
        [VERPLEEGHUIZEN],
        [GEHANDICAPTENZORGINSTELLINGEN],
        [DATE_LAST_INSERTED]
    FROM [VWSSTATIC].[NURSING_HOME_LOCATIONS_PER_REGION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[NURSING_HOME_LOCATIONS_PER_REGION])
END


## Basic

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_NURSING_HOMES_NATIONAL_ARCHIVED_20230126]
AS
    SELECT 
        [DATE_OF_REPORT_UNIX] AS DATE_UNIX,
        DBO.NO_NEGATIVE_NUMBER_I([INFECTED_NURSERY_DAILY]) AS NEWLY_INFECTED_PEOPLE, 
        DBO.NO_NEGATIVE_NUMBER_I([TOTAL_REPORTED_LOCATIONS]) AS INFECTED_LOCATIONS_TOTAL,  
        DBO.NO_NEGATIVE_NUMBER_I([DECEASED_NURSERY_DAILY]) AS DECEASED_DAILY,
        DBO.NO_NEGATIVE_NUMBER_I([TOTAL_NEW_REPORTED_LOCATIONS]) AS NEWLY_INFECTED_LOCATIONS,
        ROUND(
            100 * DBO.NO_NEGATIVE_NUMBER_F([TOTAL_REPORTED_LOCATIONS]) / T2.VERPLEEGHUIZEN
            ,2
        ) AS INFECTED_LOCATIONS_PERCENTAGE, 
        dbo.CONVERT_DATETIME_TO_UNIX(T1.DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX,
        [7D_AVERAGE_TOTAL_INFECTED]  AS NEWLY_INFECTED_PEOPLE_MOVING_AVERAGE, 
        [7D_AVERAGE_TOTAL_DECEASED] AS DECEASED_DAILY_MOViNG_AVERAGE
    FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126] T1
    LEFT JOIN (
        SELECT * FROM [VWSARCHIVE].[NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126]
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[NURSING_HOME_LOCATIONS_PER_REGION_ARCHIVED_20230126])
        ) AS T2
    ON T2.VRCODE = 'NL'
    WHERE DATE_OF_REPORT >=  '2020-02-27 00:00:00.000'
    AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
    AND T1.DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'nursing_home',
@proto_name VARCHAR(50) = 'NL',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NURSING_HOMES_NATIONAL_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'nursing_home_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## difference/nursing_home__deceased_daily

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_NURSING_HOME__DECEASED_DAILY_ARCHIVED_20230126] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126]
	WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
		AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
		AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
),

BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	LAG(DBO.NO_NEGATIVE_NUMBER_I([DECEASED_NURSERY_DAILY]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_I([DECEASED_NURSERY_DAILY]) -
		LAG(DBO.NO_NEGATIVE_NUMBER_I([DECEASED_NURSERY_DAILY]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126]
WHERE DATE_OF_REPORT >=  '2020-03-02 00:00:00.000'
AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                            FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])

)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/nursing_home__deceased_daily',
@proto_name VARCHAR(50) = 'NL',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_NURSING_HOME__DECEASED_DAILY_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/nursing_home__deceased_daily_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

## difference/nursing_home__newly_infected_people

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_NURSING_HOME__NEWLY_INFECTED_PEOPLE_ARCHIVED_20230126] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126]
	WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
		AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
		AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
),

BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	LAG(DBO.NO_NEGATIVE_NUMBER_I([INFECTED_NURSERY_DAILY]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_I([INFECTED_NURSERY_DAILY]) -
		LAG(DBO.NO_NEGATIVE_NUMBER_I([INFECTED_NURSERY_DAILY]), 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126]
WHERE DATE_OF_REPORT >=  '2020-03-02 00:00:00.000'
AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])
AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                            FROM [VWSARCHIVE].[NURSING_HOMES_TOTALS_ARCHIVED_20230126])

)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/nursing_home__newly_infected_people',
@proto_name VARCHAR(50) = 'NL',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_NURSING_HOME__NEWLY_INFECTED_PEOPLE_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/nursing_home__newly_infected_people_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

# **NURSING_HOME - VR**

---

## Tables

Archive Table

In [ ]:
-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126]'))
CREATE TABLE [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126](
	[ID] [int] NOT NULL,
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[VRCODE] [varchar](100) NULL,
	[INFECTED_NURSERY_DAILY] [int] NULL,
	[DECEASED_NURSERY_DAILY] [int] NULL,
	[TOTAL_NEW_REPORTED_LOCATIONS] [int] NULL,
	[TOTAL_REPORTED_LOCATIONS] [int] NULL,
	[INFECTED_LOCATIONS_PERCENTAGE] [decimal](16, 2) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_RANGE_START] [datetime] NULL,
	[DATE_OF_REPORTS_LAG] [datetime] NULL,
	[DATE_RANGE_START_LAG] [datetime] NULL,
	[7D_AVERAGE_TOTAL_INFECTED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_INFECTED_LAG] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED] [decimal](16, 2) NULL,
	[7D_AVERAGE_TOTAL_DECEASED_LAG] [decimal](16, 2) NULL
);
GO


Stored Procedure

In [ ]:

CREATE OR ALTER PROCEDURE [dbo].[SP_NURSING_HOMES_PER_REGION_ARCHIVED_20230126] AS
BEGIN
    TRUNCATE TABLE [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126]

    INSERT INTO [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] (
            [ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [VRCODE],
            [INFECTED_NURSERY_DAILY],
            [DECEASED_NURSERY_DAILY],
            [TOTAL_NEW_REPORTED_LOCATIONS],
            [TOTAL_REPORTED_LOCATIONS],
            [INFECTED_LOCATIONS_PERCENTAGE],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    )
    SELECT 
            [ID],
            [DATE_OF_REPORT],
            [DATE_OF_REPORT_UNIX],
            [VRCODE],
            [INFECTED_NURSERY_DAILY],
            [DECEASED_NURSERY_DAILY],
            [TOTAL_NEW_REPORTED_LOCATIONS],
            [TOTAL_REPORTED_LOCATIONS],
            [INFECTED_LOCATIONS_PERCENTAGE],
            [DATE_LAST_INSERTED],
            [DATE_RANGE_START],
            [DATE_OF_REPORTS_LAG],
            [DATE_RANGE_START_LAG],
            [7D_AVERAGE_TOTAL_INFECTED],
            [7D_AVERAGE_TOTAL_INFECTED_LAG],
            [7D_AVERAGE_TOTAL_DECEASED],
            [7D_AVERAGE_TOTAL_DECEASED_LAG]
    FROM [VWSDEST].[NURSING_HOMES_PER_REGION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[NURSING_HOMES_PER_REGION])
END


## Basic

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_NURSING_HOMES_PER_REGION_ARCHIVED_20230126] AS
    SELECT 
        DATE_OF_REPORT_UNIX AS DATE_UNIX,
        VRCODE,
        [7D_AVERAGE_TOTAL_INFECTED] AS NEWLY_INFECTED_PEOPLE_MOVING_AVERAGE, 
        [7D_AVERAGE_TOTAL_DECEASED] AS DECEASED_DAILY_MOViNG_AVERAGE,
        DBO.NO_NEGATIVE_NUMBER_I(INFECTED_NURSERY_DAILY) AS NEWLY_INFECTED_PEOPLE,
        DBO.NO_NEGATIVE_NUMBER_I(TOTAL_NEW_REPORTED_LOCATIONS) AS NEWLY_INFECTED_LOCATIONS,
        DBO.NO_NEGATIVE_NUMBER_I(TOTAL_REPORTED_LOCATIONS) AS INFECTED_LOCATIONS_TOTAL,
        DBO.NO_NEGATIVE_NUMBER_F(INFECTED_LOCATIONS_PERCENTAGE) AS INFECTED_LOCATIONS_PERCENTAGE,
        DBO.NO_NEGATIVE_NUMBER_I(DECEASED_NURSERY_DAILY) AS DECEASED_DAILY,
        dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
    FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126]

    WHERE DATE_OF_REPORT >=  '2020-02-27 00:00:00.000'
    AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126])
    AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                                FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126])
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'nursing_home',
@proto_name VARCHAR(50) = 'VR%',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NURSING_HOMES_PER_REGION_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'nursing_home_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(1024) = '*',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

## difference/nursing_home__deceased_daily

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_NURSING_HOME__DECEASED_DAILY_PER_REGION_ARCHIVED_20230126] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		VRCODE,
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK)
	WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
		AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
		AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
	GROUP BY VRCODE
),

BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX], 
	VRCODE,
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (PARTITION BY [VRCODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
    LAG(DBO.NO_NEGATIVE_NUMBER_I(DECEASED_NURSERY_DAILY), 1, NULL) OVER (PARTITION BY [VRCODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	DBO.NO_NEGATIVE_NUMBER_I(DECEASED_NURSERY_DAILY) -
		LAG(DBO.NO_NEGATIVE_NUMBER_I(DECEASED_NURSERY_DAILY), 1, NULL) OVER (PARTITION BY [VRCODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK)
WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
AND DATE_OF_REPORT <  (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,T1.VRCODE
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
	ON T1.[VRCODE] = T2.[VRCODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
;
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/nursing_home__deceased_daily',
@proto_name VARCHAR(50) = 'VR',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_NURSING_HOME__DECEASED_DAILY_PER_REGION_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/nursing_home__deceased_daily_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

## difference/nursing_home__newly_infected_people

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_NURSING_HOME__NEWLY_INFECTED_PEOPLE_PER_REGION_ARCHIVED_20230126] AS
WITH LAST_DATE_OF_REPORT AS (
SELECT
 VRCODE,
CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK)

WHERE DATE_OF_REPORT >= '2020-03-16 00:00:00.000'
AND DATE_OF_REPORT < (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
GROUP BY VRCODE
),
BASE_CTE AS (
SELECT
 DATE_OF_REPORT,
 DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
 VRCODE,
 LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (PARTITION BY [VRCODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
 LAG(DBO.NO_NEGATIVE_NUMBER_I(INFECTED_NURSERY_DAILY), 1, NULL) OVER (PARTITION BY [VRCODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
 DBO.NO_NEGATIVE_NUMBER_I(INFECTED_NURSERY_DAILY) -
 LAG(DBO.NO_NEGATIVE_NUMBER_I(INFECTED_NURSERY_DAILY), 1, NULL) OVER (PARTITION BY [VRCODE] ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
-- dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK)

WHERE DATE_OF_REPORT >= '2020-03-16 00:00:00.000'
AND DATE_OF_REPORT < (SELECT MAX(DATE_OF_REPORT) FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
)
SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
      ,T1.VRCODE
 ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
 ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
ON T1.[VRCODE] = T2.[VRCODE]
WHERE DATE_OF_REPORT = LAST_DATE_OF_REPORT
;
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'difference/nursing_home__newly_infected_people',
@proto_name VARCHAR(50) = 'VR',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] NOT LIKE '%_COLLECTION' AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_NURSING_HOME__NEWLY_INFECTED_PEOPLE_PER_REGION_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/nursing_home__newly_infected_people_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

# **NURSING_HOME - VR_COLLECTION**

---

## Basic

View

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_NURSING_HOME_REGIONS_ARCHIVED_20230126] AS 

    SELECT 
        DATE_OF_REPORT_UNIX AS DATE_UNIX,
        VRCODE, 
        NEWLY_INFECTED_PEOPLE,
        NEWLY_INFECTED_LOCATIONS,
        INFECTED_LOCATIONS_TOTAL,
        INFECTED_LOCATIONS_PERCENTAGE,
        DECEASED_DAILY,
        DATE_OF_INSERTION_UNIX
    FROM (
    SELECT
        [DATE_OF_REPORT_UNIX],
        VRCODE,
        DBO.NO_NEGATIVE_NUMBER_I(INFECTED_NURSERY_DAILY) AS NEWLY_INFECTED_PEOPLE,
        DBO.NO_NEGATIVE_NUMBER_I(TOTAL_NEW_REPORTED_LOCATIONS) AS NEWLY_INFECTED_LOCATIONS,
        DBO.NO_NEGATIVE_NUMBER_I(TOTAL_REPORTED_LOCATIONS) AS INFECTED_LOCATIONS_TOTAL,
        DBO.NO_NEGATIVE_NUMBER_F(INFECTED_LOCATIONS_PERCENTAGE) AS INFECTED_LOCATIONS_PERCENTAGE,
        DBO.NO_NEGATIVE_NUMBER_I(DECEASED_NURSERY_DAILY) AS DECEASED_DAILY,
        dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX,
        RANK() OVER (PARTITION BY VRCODE ORDER BY DATE_OF_REPORT_UNIX DESC) RANKING
    FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK)
    WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
    AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                                FROM [VWSARCHIVE].[NURSING_HOMES_PER_REGION_ARCHIVED_20230126] WITH(NOLOCK))
    AND VRCODE IS NOT NULL
    AND VRCODE != ' ') T1
    WHERE T1.RANKING =2;
GO




protos

In [ ]:
-- INACTIVATE OLD PROTO CONFIGURATION
DECLARE 
@item_name VARCHAR(256) = 'nursing_home',
@proto_name VARCHAR(50) = 'VR_COLLECTION',
@is_active INT = 0;

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [ACTIVE] = @is_active
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] LIKE @proto_name 
);

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NURSING_HOME_REGIONS_ARCHIVED_20230126',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'nursing_home_archived_20230126',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = 'VR_COLLECTION',
        @columns VARCHAR(1024) = '*',
        @layout_type_id INT = 4, -- VALUESDIRECT
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT = null,
        @config_id BIGINT = null;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;
    
SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO